# Qwen3 Tuning
for ARC AGI 2

Updates 29 July 2025:
- Now using utils from the repo.
- Updated lora r to 128, down from 256 as prob 128 is sufficient.

Updates 24 July 2025:
- Now supports calculation of metrics BUT requires validation data have code. This will be updated later to just pass grids for validation.

Updates 23 July 2025:
- Back to constant scheduler for one epoch. Note that SOAR use 3 epochs with cosine scheduler.

Updates 22 July 2025:
- Increased lora r to 256
- Added training on completions only
- Moved to cosine rather than constant scheduler. Moved from 1 to 2 epochs. Note that SOAR use 3 epochs with cosine scheduler.

In [1]:
# !git pull

In [2]:
# !git config --global user.name “RonanMcGovern”
# !git config --global user.email "78278410+RonanKMcGovern@users.noreply.github.com"

### Installation

In [3]:
# # Temporary Fix while unsloth is broken! - SHOULD BE FIXED AS OF JULY 20TH 2025
# !rm -rf /tmp/unsloth_compiled_cache
# !uv pip uninstall trl unsloth --system
# !uv pip install unsloth -qU --system
# !uv pip install trl==0.19.1 --system

In [4]:
!uv pip install vllm --system -qU

In [5]:
# INSTALLED IN THE CONTAINER IF USING the [arc-agi-2025 container on runpod](https://console.runpod.io/deploy?template=bh0rvngapk&ref=jmfkcdio)
# %%capture
# import os
# !pip install uv -qU
# !uv pip install unsloth matplotlib tensorboard -qU --system
# !export HF_HUB_ENABLE_HF_TRANSFER=1

In [6]:
from huggingface_hub import HfFolder, login

# Call this at the top of your script / notebook
if HfFolder.get_token() is None:   # no token cached or in $HF_TOKEN
    login()                        # interactive prompt

In [7]:
%load_ext tensorboard
%tensorboard --logdir ./logs --port 6006 --bind_all

### Unsloth

In [8]:
import unsloth
from unsloth import FastLanguageModel
import torch, os

model_slug = "Qwen/Qwen3-4B"
# model_slug = "julien31/Soar-qwen-7b"
# model_slug = "Qwen/Qwen2.5-Coder-7B-Instruct"
# model_slug = "Qwen/Qwen3-30B-A3B"

model_max_length = 32768 #default is ~2k for unsloth!!!

# Training AND validation batch size (incl. for autoregressive train/test example metrics calculations)
batch_size_global = 3 # use 1 for 7/8B, use 3 for 4B (can do 8 if using only 8 rows for validation)...

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_slug,
    max_seq_length = model_max_length,   # Context length - can be longer, but uses more memory
    load_in_4bit = False,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    cache_dir = '',
    fast_inference = True, # allows for vLLM generation during evaluation
    max_lora_rank=128, 
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-30 13:45:04 [__init__.py:235] Automatically detected platform cuda.
Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.7.11: Fast Qwen3 patching. Transformers: 4.54.1. vLLM: 0.10.0.
   \\   /|    NVIDIA H200. Num GPUs = 1. Max memory: 139.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B with actual GPU utilization = 49.79%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 139.72 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 32768. Num Sequences = 36

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 07-30 13:45:15 [default_loader.py:262] Loading weights took 1.11 seconds
INFO 07-30 13:45:15 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 07-30 13:45:16 [gpu_model_runner.py:1892] Model loading took 8.0059 GiB and 1.449210 seconds
INFO 07-30 13:45:27 [backends.py:530] Using cache directory: /root/.cache/vllm/torch_compile_cache/4cd45fbc22/rank_0_0/backbone for vLLM's torch.compile
INFO 07-30 13:45:27 [backends.py:541] Dynamo bytecode transform time: 10.99 s
INFO 07-30 13:45:37 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 9.225 s
INFO 07-30 13:45:42 [monitor.py:34] torch.compile takes 10.99 s in total
INFO 07-30 13:45:43 [gpu_worker.py:255] Available KV cache memory: 58.55 GiB
INFO 07-30 13:45:43 [kv_cache_utils.py:833] GPU KV cache size: 426,304 tokens
INFO 07-30 13:45:43 [kv_cache_utils.py:837] Maximum concurrency for 32,768 tokens per request: 13.01x
INFO 07-30 13:45:43 [vllm_utils.py:641] Unsloth: Running patched vLLM v1 `

Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:19<00:00,  3.40it/s]

INFO 07-30 13:46:03 [gpu_model_runner.py:2485] Graph capturing finished in 20 secs, took 1.00 GiB
INFO 07-30 13:46:03 [vllm_utils.py:648] Unsloth: Patched vLLM v1 graph capture finished in 20 secs.


INFO 07-30 13:46:04 [core.py:193] init engine (profile, create kv cache, warmup model) took 48.33 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']


In [9]:
print(model.max_seq_length)

32768


In [10]:
# # Print a summary of the transformer layers and key dimensions
# for i, block in enumerate(model.model.layers):
#     attn = block.self_attn
#     mlp = block.mlp

#     print(f"Layer {i}:")
#     print(f"  Attention:")
#     print(f"    q_proj: {attn.q_proj.weight.shape}")
#     print(f"    k_proj: {attn.k_proj.weight.shape}")
#     print(f"    v_proj: {attn.v_proj.weight.shape}")
#     print(f"    out_proj: {attn.o_proj.weight.shape}")
#     print(f"  MLP:")
#     print(f"    fc1: {mlp.gate_proj.weight.shape}")
#     print(f"    fc2: {mlp.up_proj.weight.shape}")
#     print(f"    fc3: {mlp.down_proj.weight.shape}")
#     print()

In [11]:
# print(model)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128. could consider 128.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                     ],
    lora_alpha = 64,  # Best to choose alpha = rank or rank*2. EXCEPT if using rslora, in which case set it as sqrt(max matrix dimension). 64 is good for Qwen 4B
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.7.11 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [13]:
print(tokenizer.padding_side)

right


In [ ]:
# Import utils using standard project root detection
from pathlib import Path
import sys

# Find project root by looking for pyproject.toml
project_root = next(
    (parent for parent in [Path.cwd()] + list(Path.cwd().parents) 
     if (parent / "pyproject.toml").exists()), 
    Path.cwd()
)

# Add llm_python directory to path (where utils is located)
llm_python_dir = project_root / "llm_python"
sys.path.insert(0, str(llm_python_dir))

print(f"📁 Project root: {project_root}")
print(f"📁 Looking for utils in: {llm_python_dir}")

from utils.task_loader import TaskLoader
from utils.scoring import GridScorer, ProgramExecutor
from utils.prompt_utils import create_arc_prompt, extract_python_code
from utils.metrics_utils import calculate_task_metrics, format_metrics_display, metrics_to_percentages
from utils.timeout_utils import execute_with_timeout
from utils.transduction import is_transduction_cheating
from utils.prompt_loader import PromptLoader

# Initialize utility instances
prompt_loader = PromptLoader()
scorer = GridScorer()
print("✅ Utils imported and initialized successfully")

📁 Project root: /workspace/arc-agi-2025
📁 Looking for utils in: /workspace/arc-agi-2025/llm_python
✅ Utils imported and initialized successfully


<a name="Data"></a>
### Data Prep

In [ ]:
from pathlib import Path
import json
from typing import Optional
from datasets import load_dataset, DatasetDict

# ---------------------------------------------------------------------
# Config (examples)
# ---------------------------------------------------------------------

# Set max_rows flag to limit train size. None for all
max_rows = None # for all rows
max_validation_rows =32

# # CASE 1: single slug with both splits
# train_slug = "Trelis/synth_arc-agi-1_all_training_20250724_131808"
# val_slug = None

# CASE 2: two different slugs
train_slug = "Trelis/soar-20250729_114431-1600"
val_slug   = "Trelis/grids_only_arc-agi-1_all_evaluation_20250729_102313"

enable_thinking = False  # See note in original code

# ---------------------------------------------------------------------
# Prompt management using utils (replacing hard-coded prompts)
# ---------------------------------------------------------------------

# Use prompt_loader to get SOAR prompts from utils
SYSTEM_PROMPT = prompt_loader.get_system_message("soar")
INITIAL_TURN_PROMPT = prompt_loader.get_initial_turn_prompt("soar")

print(f"✅ Using SOAR prompts from utils:")
print(f"   System prompt: {len(SYSTEM_PROMPT)} chars")
print(f"   Initial turn prompt: {len(INITIAL_TURN_PROMPT)} chars")

# Custom formatting functions removed - using create_arc_prompt from utils instead

def hf_dataset_to_chat_dataset(dataset_slug: str, split: str = "train"):
    """
    Convert a HF split into the chat/prompt format using utils.
    """
    ds = load_dataset(dataset_slug, split=split, keep_in_memory=True)

    def select_output(example, output_key, predicted_key):
        """
        Select between regular output and predicted output based on availability and non-zero values.
        """
        # Check if predicted output exists and is non-zero
        if predicted_key in example and example[predicted_key]:
            predicted = example[predicted_key]
            # Check if any predicted output is non-zero (has any non-zero cells)
            if any(any(any(cell != 0 for cell in row) for row in grid) for grid in predicted):
                return predicted
        
        # Fall back to regular output
        return example[output_key]

    def create_chat_messages(example):
        # Select appropriate outputs (predicted if available and non-zero, otherwise regular)
        train_outputs = select_output(example, "train_output", "predicted_train_output")
        test_outputs = select_output(example, "test_output", "predicted_test_output")
        
        # Create task_data in the format expected by create_arc_prompt
        task_data = {
            'train': [{'input': inp, 'output': out} 
                     for inp, out in zip(example["train_input"], train_outputs)],
            'test': [{'input': inp, 'output': out} 
                     for inp, out in zip(example["test_input"], test_outputs)]  # All test examples!
        }
        
        # Use create_arc_prompt from utils (matches run_arc_tasks_soar.py)
        system_content, user_content = create_arc_prompt(task_data, prompt_loader, "soar")

        assistant_content = ""
        if enable_thinking and example.get("reasoning", "").strip():
            assistant_content += f"<think>{example['reasoning'].strip()}</think>"
        assistant_content += f"```python\n{example['code']}\n```"

        messages = [
            {"role": "system", "content": system_content},
            {"role": "user",   "content": user_content},
            {"role": "assistant", "content": assistant_content},
        ]

        prompt_messages = [
            {"role": "system", "content": system_content},
            {"role": "user",   "content": user_content},
        ]

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
            enable_thinking = enable_thinking
        )

        prompt_text = tokenizer.apply_chat_template(
            prompt_messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking = enable_thinking
        )

        return {
            "messages": messages,
            "text": text,
            "prompt": prompt_text,
            "train_input": example["train_input"],
            "train_output": example["train_output"],
            "test_input": example["test_input"],
            "test_output": example["test_output"],
            "reasoning": example.get("reasoning", ""),
            "code": example["code"],
            "task_id": example.get("task_id", ""),
        }

    ds = ds.map(create_chat_messages, desc=f"build chat + prompt fields ({split})")
    return ds

def build_dataset(train_slug: str,
                  val_slug: Optional[str] = None,
                  train_split: str = "train",
                  val_split: str = "validation") -> DatasetDict:
    """
    Build a DatasetDict with 'train' and 'validation' keys.
    - If val_slug is None, both splits are loaded from train_slug.
    - Otherwise, load train_split from train_slug and val_split from val_slug.
    """
    
    # Load and filter
    train_ds = hf_dataset_to_chat_dataset(train_slug, split=train_split)
    if max_rows:
        train_ds = train_ds.select(range(min(len(train_ds), max_rows)))
    
    # Validation logic
    if val_slug is None:
        try:
            val_ds = hf_dataset_to_chat_dataset(train_slug, split=val_split)
        except Exception as e:
            raise ValueError(
                f"Could not load split '{val_split}' from '{train_slug}'. "
                f"Pass an explicit val_slug or choose a valid split.\nOriginal error: {e}"
            )
    else:
        val_ds = hf_dataset_to_chat_dataset(val_slug, split="train")
    
    # Corrected slice
    if max_validation_rows:
        val_ds = val_ds.select(range(min(len(val_ds), max_validation_rows)))
    elif max_rows:
        val_ds = val_ds.select(range(min(len(val_ds), max_rows)))
    
    return DatasetDict(train=train_ds, validation=val_ds)

# ---------------------------------------------------------------------
# Build the dataset
# ---------------------------------------------------------------------
data = build_dataset(train_slug, val_slug)  # val_slug may be None 

✅ Using SOAR prompts from utils:
   System prompt: 129 chars
   Initial turn prompt: 990 chars


Parameter 'function'=<function hf_dataset_to_chat_dataset.<locals>.create_chat_messages at 0x7b357bf8c2c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


build chat + prompt fields (train):   0%|          | 0/1600 [00:00<?, ? examples/s]

build chat + prompt fields (train):   0%|          | 0/400 [00:00<?, ? examples/s]

In [16]:
# Validation split
val_ids = [ex["task_id"] for ex in data["validation"]]
assert all(val_ids), "❌ some validation rows are missing task_id"
assert len(val_ids) == len(set(val_ids)), "❌ duplicate task_id in validation slice"

In [17]:
# # print(data["train"][0])
# print(data["train"][0]['prompt'])

In [18]:
# print(data["train"][0]['text'])

In [19]:
print(data["validation"][0]['text'])

<|im_start|>system
You are an AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks by reasoning and generating Python code.<|im_end|>
<|im_start|>user
You are an AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks by generating Python code.
Your goal is to analyze input-output grid pairs. The outputs were produced by applying a transformation rule to the inputs. Implement the transformation rules as a Python function.
You should only write the implemented the transformation in code.
You must write code in triple backticks (```python and then ```). You must write a function called 'transform' which takes a single argument, the input grid as 'list[list[int]]', and returns the transformed grid (also as 'list[list[int]]').
You should make sure that you implement a version of the transformation which works in general (at least for all given input-output pairs and test input pairs).
The number in the input grid can be mapped to the following

Let's see the structure of both datasets:

In [20]:
import numpy as np
from statistics import median

def length_stats(dataset, name=""):
    """
    Return min / median / max tokenised length for a 🤗 Dataset split that has a
    single 'text' column. Uses the same tokenizer already in memory.
    """
    # Tokenise in batches → list of list[int] → list[int] lengths
    lengths = dataset.map(
        lambda batch: {
            "len": [len(ids) for ids in tokenizer(batch["text"],
                                                  add_special_tokens=False
                                                 )["input_ids"]]
        },
        batched=True,
        remove_columns=dataset.column_names,   # drop 'text'
        keep_in_memory=True,
    )["len"]

    print(f"{name:>11}:  min={min(lengths):>4}  "
          f"median={int(median(lengths)):>4}  max={max(lengths):>4}")

# ── run for both splits ────────────────────────────────────────────────────────
length_stats(data["train"],       "train")
length_stats(data["validation"],  "validation")


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

      train:  min= 518  median=2303  max=19056


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

 validation:  min= 631  median=3561  max=15051


In [21]:
from collections import Counter
from tqdm.auto import tqdm

# the same kwargs TRL uses internally
tok_kwargs = dict(
    padding=False,            # TRL pads in the collator
    truncation=True,          # applies max_length if set in SFTConfig (defaults to 2048)
    max_length=32768,
    add_special_tokens=True,
)

empty_rows   = []
bos_only     = []
length_hist  = Counter()

for idx, text in tqdm(enumerate(data["validation"]["text"]), total=len(data["validation"])):
    ids = tokenizer(text, **tok_kwargs)["input_ids"]
    length_hist[len(ids)] += 1

    # what vLLM actually sees after we strip specials/whitespace
    visible = tokenizer.decode(
        ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    ).strip()

    if len(ids) == 0:
        empty_rows.append((idx, "<zero tokens>"))
    elif not visible:
        bos_only.append((idx, ids[:4]))       # keep first few token ids for quick glance

print("Histogram of tokenised lengths:", length_hist.most_common(10))
print("\nROWS WITH ZERO TOKENS:", empty_rows[:5])
print("\nROWS THAT BECAME BOS‑ONLY (after special‑token strip):")
for idx, tok_snip in bos_only[:5]:
    print(f"  row {idx}: first ids {tok_snip}")
    print("     raw snippet:", repr(data['validation']['text'][idx][:120]))


  0%|          | 0/32 [00:00<?, ?it/s]

Histogram of tokenised lengths: [(2091, 3), (631, 1), (5347, 1), (4259, 1), (15051, 1), (8511, 1), (1163, 1), (5319, 1), (2725, 1), (9852, 1)]

ROWS WITH ZERO TOKENS: []

ROWS THAT BECAME BOS‑ONLY (after special‑token strip):


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).

In [22]:
# print(help(model.fast_generate))

In [23]:
from datetime import datetime
import re

# Extract date and time using regex
print("Extract training set date and time as dataset identifiers")

# Try pattern 1: timestamp at end (original pattern)
match = re.search(r'(\d{8}_\d{6})$', train_slug)
if match:
    timestamp = match.group(1)
    date_str = timestamp[:8]
    time_str = timestamp[9:]
    print(f"Date: {date_str} (YYYYMMDD)")
    print(f"Time: {time_str} (HHMMSS)")
else:
    # Try pattern 2: SOAR dataset format (soar-YYYYMMDD_HHMMSS-rows)
    match = re.search(r'soar-(\d{8}_\d{6})-\d+', train_slug)
    if match:
        timestamp = match.group(1)
        date_str = timestamp[:8]
        time_str = timestamp[9:]
        print(f"Date: {date_str} (YYYYMMDD)")
        print(f"Time: {time_str} (HHMMSS)")
    else:
        print("No timestamp found.")
        date_str = "unknown"
        time_str = "unknown"

run_name = f"{model_slug.split('/')[-1]}-ds{date_str}_{time_str}-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
print(f"Run name will be {run_name}")

Extract training set date and time as dataset identifiers
Date: 20250729 (YYYYMMDD)
Time: 114431 (HHMMSS)
Run name will be Qwen3-4B-ds20250729_114431-20250730-134621


In [24]:
# import re, copy, traceback, numpy as np, torch, signal
# from types import FunctionType
# from transformers.trainer_utils import EvalPrediction

# # ---- Code extraction using utils (replacing custom regex)
# # Note: extract_python_code function now imported from utils.prompt_utils
# # No need for custom regex patterns - using the tested utils implementation

In [25]:
import torch, subprocess, os, gc, time

def _print_gpu(prefix=""):
    alloc = torch.cuda.memory_allocated() / 2**20  # MiB
    reserved = torch.cuda.memory_reserved() / 2**20
    print(f"{prefix}CUDA‑alloc={alloc:.0f} MiB | reserved={reserved:.0f} MiB")

def _nvidia_smi():
    try:
        smi = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=memory.used,memory.free",
             "--format=csv,noheader,nounits"]).decode().strip()
        print("nvidia-smi (used/free MiB):", smi)
    except Exception:
        pass  # nvidia-smi not always available


In [26]:
# vllm_memory_callback.py
import torch, gc, time, subprocess
from contextlib import suppress
from transformers import TrainerCallback

class VllmMemoryCallback(TrainerCallback):
    # *args covers (args, state, control)
    # **kwargs so we accept model=..., metrics=..., etc.
    def on_evaluate(self, *args, **kwargs):
        # ---------------- snapshots ----------------
        used_free = subprocess.check_output(
            ["nvidia-smi",
             "--query-gpu=memory.used,memory.free",
             "--format=csv,noheader,nounits"]
        ).decode().strip()
        print(f"[MEM‑before] alloc={torch.cuda.memory_allocated()//2**20}MiB "
              f"reserved={torch.cuda.memory_reserved()//2**20}MiB  smi {used_free}")

        # ---------------- flush vLLM + torch ----------------
        with suppress(Exception):
            model = kwargs["model"]                # grab from kw
            model.llm_engine.reset()               # KV cache
            model.llm_engine._allocator._memory_pool.clear()  # CUDA blocks
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize(); time.sleep(0.2)

        used_free = subprocess.check_output(
            ["nvidia-smi",
             "--query-gpu=memory.used,memory.free",
             "--format=csv,noheader,nounits"]
        ).decode().strip()
        print(f"[MEM‑after ] alloc={torch.cuda.memory_allocated()//2**20}MiB "
              f"reserved={torch.cuda.memory_reserved()//2**20}MiB  smi {used_free}")

# Smaller batch size support
import torch
from trl import SFTTrainer
from vllm import SamplingParams

MAX_NEW_TOKENS = 1000  # one place to tune

class VllmSFTTrainer(SFTTrainer):
    def prediction_step(self, model, inputs, prediction_loss_only=False, ignore_keys=None):
        print(f"[vLLM predict] batch size = {inputs['input_ids'].shape[0]}")
        ids = inputs["input_ids"].to(self.args.device)

        # 1) Find prompts up to response_part
        response_tag_ids = self.tokenizer(response_tag, add_special_tokens=False)["input_ids"]
        def _find_sublist(lst, sublst):
            L, M = len(lst), len(sublst)
            for i in range(L - M + 1):
                if lst[i : i + M] == sublst:
                    return i
            raise ValueError

        prompts = []
        for batch_idx, row in enumerate(ids):
            row_ids = row.tolist()
            try:
                start = _find_sublist(row_ids, response_tag_ids)
                cut   = start + len(response_tag_ids)
            except ValueError:
                cut = len(row_ids)
            text = self.tokenizer.decode(
                row_ids[:cut],
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            ).strip()
            if not text:
                print(f"[prompt‑dbg] row {batch_idx}: empty → using response_part only")
                text = response_tag
            prompts.append(text)

        # 2) Generate with vLLM
        params = SamplingParams(temperature=0.1, min_p=0.05, max_tokens=MAX_NEW_TOKENS)
        with torch.no_grad():
            outs = model.fast_generate(prompts, sampling_params=params)

        # 3) Convert each output to a 1D LongTensor
        gen_seqs = [
            torch.tensor(o.outputs[0].token_ids, dtype=torch.long)
            for o in outs
        ]

        # 4) Manually pad or truncate *every* sequence to EXACTLY MAX_NEW_TOKENS
        pad_id = self.tokenizer.pad_token_id
        batch_size = len(gen_seqs)
        # allocate [batch, MAX_NEW_TOKENS] filled with pad_id
        fixed = gen_seqs[0].new_full((batch_size, MAX_NEW_TOKENS), pad_id)
        for i, seq in enumerate(gen_seqs):
            length = min(seq.size(0), MAX_NEW_TOKENS)
            fixed[i, :length] = seq[:length]

        gen_ids = fixed.cpu()  # shape = [batch_size, MAX_NEW_TOKENS]

        # 5) Return in HF–Trainer’s expected format
        return (None, gen_ids, inputs.get("labels"))

# import torch
# from trl import SFTTrainer
# from vllm import SamplingParams
# from torch.nn.utils.rnn import pad_sequence
# from vllm.inputs import TokensPrompt

# MAX_NEW_TOKENS = 1000                   # one place to tune

# class VllmSFTTrainer(SFTTrainer):
#     def prediction_step(self, model, inputs, prediction_loss_only=False, ignore_keys=None):
#         print(f"[vLLM predict] batch size = {inputs['input_ids'].shape[0]}")
#         ids = inputs["input_ids"].to(self.args.device)

#         # 1) Precompute the token‑ID sequence for your response_part tag
#         response_tag_ids = self.tokenizer(
#             response_tag,
#             add_special_tokens=False
#         )["input_ids"]

#         # helper to find the first occurrence of that subsequence
#         def _find_sublist(lst, sublst):
#             L, M = len(lst), len(sublst)
#             for i in range(L - M + 1):
#                 if lst[i : i + M] == sublst:
#                     return i
#             raise ValueError

#         prompts = []
#         for batch_idx, row in enumerate(ids):
#             row_ids = row.tolist()

#             # 2) locate the end of the response_part marker
#             try:
#                 start = _find_sublist(row_ids, response_tag_ids)
#                 cut   = start + len(response_tag_ids)
#             except ValueError:
#                 # fallback: use entire attention_mask slice or full row
#                 # Here we simply take the whole row
#                 cut = len(row_ids)

#             # 3) decode tokens up to that point
#             prompt_ids = row_ids[:cut]
#             text = self.tokenizer.decode(
#                 prompt_ids,
#                 skip_special_tokens=True,
#                 clean_up_tokenization_spaces=True
#             ).strip()

#             # (optional) debug if somehow empty
#             if not text:
#                 print(f"[prompt‑dbg] row {batch_idx} empty after slicing → using response_part only")
#                 text = response_tag  # at least the assistant‑BOS marker

#             prompts.append(text)

#         # … then your SamplingParams + model.fast_generate(prompts) as before …
#         params = SamplingParams(temperature=0.1, min_p=0.05, max_tokens=MAX_NEW_TOKENS)
#         with torch.no_grad():
#             outs = model.fast_generate(prompts, sampling_params=params)

#         gen_ids = [
#             torch.tensor(o.outputs[0].token_ids, dtype=torch.long)
#             for o in outs
#         ]
#         gen_ids = pad_sequence(gen_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id).cpu()

#         return (None, gen_ids, inputs.get("labels"))

In [ ]:
from pprint import pprint

DEBUG = False           # ← flip this or pass debug=True below

def compute_arc_metrics(eval_pred, *, debug: bool = DEBUG):
    """
    • Supports any number of test grids.
    • Writes a reliable 'all_test_correct' flag.
    • Tracks 'code_ran' for the downstream min‑1‑code‑runs metric.
    • Prints rich per‑task debug when debug=True.
    
    NOTE: This function uses train_output and test_output for evaluation metrics.
    It assumes these represent the programs' outputs (either original or predicted).
    """
    print("⚠️  METRICS WARNING: Using train_output and test_output for evaluation. Assuming these represent the programs' outputs.")
    
    preds, _ = eval_pred
    
    # preds is now [batch, seq_len] int32/int64
    generated_texts = tokenizer.batch_decode(preds, skip_special_tokens=True)

    scorer   = GridScorer()
    executor = ProgramExecutor(timeout=0.5, executor_type="unrestricted")
    raw_dataset = trainer.eval_dataset

    results = []

    for idx, generated_text in enumerate(generated_texts):
        ex   = raw_dataset[idx]
        code = extract_python_code(generated_text)

        if debug:
            print(f"\n=== ARC item {idx} ===")
            # print("Generated text:\n----------------")
            # print(generated_text.strip() or "<EMPTY>")
            # print("\nExtracted code:\n---------------")
            # print(code or "<NO CODE>")

        train_results = []
        test_preds    = []        # list of grids (may stay empty)
        all_test_correct = None   # sentinel
        any_timeout   = False
        code_ran      = False     # becomes True once *one* grid is produced

        # ---------------- TRAIN CASES -----------------
        if code:
            for t_i, (tr_in, tr_out) in enumerate(zip(ex["train_input"], ex["train_output"])):
                try:
                    g, err, timed = executor.execute_program_with_timeout(code, tr_in)
                    if timed: any_timeout = True
                    if g is not None:
                        code_ran   = True
                        is_correct = scorer.score_grid(g, tr_out)["correct"]
                    else:
                        is_correct = False
                except Exception:
                    g = None
                    is_correct = False

                train_results.append({"pred": g, "correct": is_correct})

                if debug:
                    print(f"\nTrain case {t_i}: correct={is_correct}")
                    pprint({"pred": g, "gt": tr_out})

        # ---------------- TEST CASES ------------------
        if code and ex["test_input"] and ex["test_output"]:
            for c_i, (te_in, te_out) in enumerate(zip(ex["test_input"], ex["test_output"])):
                try:
                    g, err, timed = executor.execute_program_with_timeout(code, te_in)
                    if timed: any_timeout = True
                    if g is not None:
                        code_ran   = True
                        is_correct = scorer.score_grid(g, te_out)["correct"]
                    else:
                        is_correct = False
                except Exception:
                    g = None
                    is_correct = False

                test_preds.append(g)
                all_test_correct = (
                    is_correct if all_test_correct is None
                    else (all_test_correct and is_correct)
                )

                if debug:
                    print(f"\nTest case {c_i}: correct={is_correct}")
                    # pprint({"pred": g, "gt": te_out})

        if all_test_correct is None:          # no test grids produced
            all_test_correct = False

        # ---------------- PACKAGE ATTEMPT -------------
        attempt_details = [{
            "program"          : code or "",
            "code_ran"         : code_ran,
            "test_pred"        : test_preds,           # list (possibly empty)
            "all_test_correct" : all_test_correct,     # boolean
            "train_results"    : train_results,
            "api_success"      : True,
            "api_timeout"      : any_timeout,
            "output_tokens"    : len(generated_text.split()) if generated_text else 0,
        }]

        task_data = {
            "train": [{"input": i, "output": o} for i, o in zip(ex["train_input"], ex["train_output"])],
            "test" : [{"input": i, "output": o} for i, o in zip(ex["test_input"],  ex["test_output"])],
        }

        results.append({"task_data": task_data, "attempt_details": attempt_details})

    # -------------- AGGREGATE ------------------------
    metrics = calculate_task_metrics(results, max_tokens=MAX_NEW_TOKENS)

    tot   = max(1, metrics["total"])
    tresp = max(1, metrics["total_responses"])

    return {
        "all_test_correct"  : metrics["all_test_correct"]  / tot,
        "all_train_correct" : metrics["all_train_correct"] / tot,
        "min1_train_correct": metrics["min1_train_correct"]/ tot,
        "min1_transductive" : metrics["min1_transductive"] / tot,
        "min1_code_success" : metrics["min1_code_success"] / tot,
        "max_length_rate"   : metrics["max_length_responses"] / tresp,
        "timeout_rate"      : metrics["timeout_responses"]    / tresp,
        "total_tasks"       : tot,
        "total_responses"   : tresp,
    }

In [28]:
# Update the trainer configuration to use the fixed custom metrics
from trl import SFTTrainer, SFTConfig

# make sure the attr exists so Unsloth can safely delete it
setattr(model, "_flag_for_generation", True)

trainer = VllmSFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=batch_size_global,
        # per_device_eval_batch_size = batch_size_global,
        per_device_eval_batch_size = len(data["validation"]), # to run all validation data through vllm, may affect original memory allocation and result in smaller training batch sizes
        gradient_accumulation_steps=int(32 / batch_size_global),
        warmup_steps=5,
        eval_steps=0.1,
        do_eval=True,
        eval_strategy="steps",
        num_train_epochs=3,
        # max_steps=3,
        learning_rate=1e-4,
        logging_steps=0.05,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="constant",
        seed=3407,
        report_to="tensorboard",
        logging_dir=f"./logs/{run_name}",
        remove_unused_columns=True,
        save_strategy="steps",
        save_steps=0.2,
        save_total_limit=3,
        prediction_loss_only=False
    ),
    compute_metrics=compute_arc_metrics,
    callbacks=[VllmMemoryCallback()],
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1600 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/32 [00:00<?, ? examples/s]

In [29]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H200. Max memory = 139.719 GB.
70.02 GB of memory reserved.


In [30]:
# We should consider training on completions only!!! which means the response part for the xentropy.

In [31]:
from unsloth.chat_templates import train_on_responses_only # or run the code above if not using unsloth

TEMPLATES = {
    "llama": (
        "<|start_header_id|>user<|end_header_id|>\n\n",
        "<|start_header_id|>assistant<|end_header_id|>\n\n",
    ),
    "gemma": (
        "<start_of_turn>user\n",
        "<start_of_turn>model\n",
    ),
    "qwen-coder": (
        "<|im_start|>user\n",
        "<|im_start|>assistant\n", # this is actually how you properly allow the model to keep reasoning!
    ),
    "qwen": (
        "<|im_start|>user\n",
        "<|im_start|>assistant\n<think>\n\n</think>\n\n", # this is actually how you properly allow the model to keep reasoning!
    ),
    "mistral": (
        "[INST]",
        "[/INST]",
    )
}

# instruction_tag, response_tag = TEMPLATES["qwen-coder"]   # ← change if needed and comment out below

model_slug_lower = model_slug.lower()

if "qwen" in model_slug_lower:
    if "coder" in model_slug_lower:
        instruction_tag, response_tag = TEMPLATES["qwen-coder"]
    elif "soar-qwen" in model_slug_lower:
        instruction_tag, response_tag = TEMPLATES["qwen-coder"]
    else:
        instruction_tag, response_tag = TEMPLATES["qwen"]
else:
    raise ValueError(f"Unsupported model slug for Qwen template: {model_slug}")

# TO SUPPORT REASONING, WE NEED TO DYNAMICALLY APPLY THE RIGHT MASKING, NOT YET IMPLEMENTED
# masks everything between the instruction_part and response_part
trainer = train_on_responses_only(
    trainer,
    instruction_part = instruction_tag,
    response_part = response_tag,
    # force_match=False # comment out to set true for a cleaner masking
)

Map (num_proc=192):   0%|          | 0/1600 [00:00<?, ? examples/s]

num_proc must be <= 32. Reducing num_proc to 32 for dataset of size 32.


Map (num_proc=32):   0%|          | 0/32 [00:00<?, ? examples/s]

In [32]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

"<|im_start|>system\nYou are an AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks by reasoning and generating Python code.<|im_end|>\n<|im_start|>user\nYou are an AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks by generating Python code.\nYour goal is to analyze input-output grid pairs. The outputs were produced by applying a transformation rule to the inputs. Implement the transformation rules as a Python function.\nYou should only write the implemented the transformation in code.\nYou must write code in triple backticks (```python and then ```). You must write a function called 'transform' which takes a single argument, the input grid as 'list[list[int]]', and returns the transformed grid (also as 'list[list[int]]').\nYou should make sure that you implement a version of the transformation which works in general (at least for all given input-output pairs and test input pairs).\nThe number in the input grid can be mapped to the 

In [33]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[0]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [34]:
# # if you need to reimport libraries
# import importlib
# import utils.metrics_utils as mu

# importlib.reload(mu)

Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,600 | Num Epochs = 3 | Total steps = 150
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 264,241,152 of 4,286,709,248 (6.16% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,600 | Num Epochs = 3 | Total steps = 201
O^O/ \_/ \    Batch size per device = 3 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (3 x 8 x 1) = 24
 "-____-"     Trainable parameters = 264,241,152 of 4,286,709,248 (6.16% trained)


Step,Training Loss,Validation Loss,All Test Correct,All Train Correct,Min1 Train Correct,Min1 Transductive,Min1 Code Runs,Max Length Rate,Timeout Rate,Total Tasks,Total Responses
21,0.164000,No log,0.000000,0.000000,0.000000,0.031250,0.812500,0.000000,0.000000,32,32
42,0.232000,No log,0.000000,0.000000,0.000000,0.031250,0.781250,0.000000,0.000000,32,32


...

[vLLM predict] batch size = 32


Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



=== ARC item 0 ===

Train case 0: correct=False
{'gt': [[8, 6, 8, 6, 8, 6],
        [6, 4, 6, 4, 6, 4],
        [6, 8, 6, 8, 6, 8],
        [4, 6, 4, 6, 4, 6],
        [8, 6, 8, 6, 8, 6],
        [6, 4, 6, 4, 6, 4]],
 'pred': [[8, 8, 0, 6, 6, 0],
          [8, 8, 0, 6, 6, 0],
          [0, 0, 0, 0, 0, 0],
          [6, 6, 0, 4, 4, 0],
          [6, 6, 0, 4, 4, 0],
          [0, 0, 0, 0, 0, 0]]}

Train case 1: correct=False
{'gt': [[7, 9, 7, 9, 7, 9],
        [4, 3, 4, 3, 4, 3],
        [9, 7, 9, 7, 9, 7],
        [3, 4, 3, 4, 3, 4],
        [7, 9, 7, 9, 7, 9],
        [4, 3, 4, 3, 4, 3]],
 'pred': [[7, 7, 0, 9, 9, 0],
          [7, 7, 0, 9, 9, 0],
          [0, 0, 0, 0, 0, 0],
          [4, 4, 0, 3, 3, 0],
          [4, 4, 0, 3, 3, 0],
          [0, 0, 0, 0, 0, 0]]}

Test case 0: correct=False

=== ARC item 1 ===

Train case 0: correct=False
{'gt': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2,

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
print(trainer_stats)

In [ ]:
# Check a few examples
for i in range(2):
    example = data["train"][i]
    print(f"\nExample {i} text length: {len(example['text'])}")
    print(f"Last 200 chars: {example['text'][-200:]}")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [ ]:
# messages = [
#     {"role" : "system", "content" : "You are an expert at solving abstract reasoning puzzles. Write clean, efficient Python code."},
#     {"role" : "user", "content" : "You are solving an ARC (Abstraction and Reasoning Corpus) task. I will show you training examples with input and output grids, plus a test input grid. Your task is to: 1. **Analyze the training examples** to discover patterns that map input grids to output grids 2. **Write a Python program** that implements your best understanding of the transformation 3. **DO NOT predict or generate the test output** - your job is only to write the transformation program 4. **Attempt a solution** - even if the pattern isn't completely clear, provide your best hypothesis 5. **Do not repeat the same transformation** - if you have already tried a transformation, do not repeat it. **IMPORTANT: Your transformation must always produce a 3×3 output grid.** The test input is shown for context so you understand what type of grid your program will eventually process. Focus on learning patterns from training examples and writing code that captures your understanding. Training Examples: Example 1: Input: 6 6 6 7 7 7 0 0 6 0 0 6 0 0 6 6 0 0 6 6 0 0 0 0 6 0 0 0 7 0 0 7 0 0 6 7 6 0 7 0 6 0 0 7 6 0 0 0 0 7 0 0 6 6 0 0 0 6 7 6 7 0 7 2 2 2 2 0 7 0 7 7 0 7 0 0 6 7 0 6 0 7 0 2 2 2 2 7 6 7 0 0 6 7 6 0 7 0 6 0 7 0 0 2 2 2 2 7 0 0 0 0 6 7 0 0 0 0 0 0 6 6 6 2 2 2 2 0 7 0 0 9 9 9 7 7 0 7 7 0 7 0 0 0 7 0 0 7 6 0 6 9 9 9 7 0 6 0 0 0 7 6 0 6 6 7 0 6 0 6 7 9 9 9 6 0 0 0 0 0 0 0 7 7 6 0 7 6 6 7 6 9 9 9 0 0 0 0 0 0 0 7 7 6 0 0 0 0 7 0 6 0 6 0 0 7 0 7 0 0 0 6 7 7 0 7 7 7 0 0 0 6 6 6 0 0 0 0 6 6 6 7 7 0 7 6 0 6 0 0 0 0 0 6 0 0 0 0 0 7 7 6 6 0 6 7 0 6 0 6 7 6 0 0 6 7 0 0 7 6 6 0 0 6 0 7 4 4 4 4 4 0 0 7 6 0 6 0 0 0 7 0 7 0 0 0 4 4 4 4 4 7 0 7 6 0 0 0 0 7 6 6 0 0 0 0 4 4 4 4 4 0 0 0 0 0 0 6 0 0 6 7 6 6 6 0 4 4 4 4 4 6 7 7 6 7 0 0 0 6 7 0 0 0 6 6 4 4 4 4 4 6 0 6 0 0 0 0 6 7 0 0 7 7 6 0 0 6 7 6 6 0 6 0 6 0 7 7 0 0 Output: 0 0 0 0 0 0 0 0 0 Example 2: Input: 8 8 8 8 8 0 1 0 1 1 8 1 1 1 0 0 0 0 0 1 1 0 0 0 0 8 1 1 0 0 0 8 0 5 5 5 5 5 8 1 0 0 0 0 0 8 1 0 0 8 1 1 1 5 5 5 5 5 8 0 1 8 0 1 8 0 0 8 8 8 8 1 8 5 5 5 5 5 1 0 0 8 0 9 9 9 9 8 0 0 0 0 0 5 5 5 5 5 8 0 8 1 8 9 9 9 9 8 1 1 0 1 1 0 8 0 8 8 0 8 0 0 0 9 9 9 9 0 1 1 8 8 3 3 8 1 1 0 0 1 8 1 1 8 1 8 0 1 0 0 0 3 3 3 1 0 8 1 8 8 0 1 8 8 1 1 0 8 8 3 3 3 3 3 8 0 0 8 1 0 0 1 1 0 1 0 0 0 8 3 3 3 3 3 1 1 8 8 1 0 8 0 8 0 8 0 0 0 0 3 3 3 3 3 1 1 1 0 8 8 0 0 0 0 8 1 1 1 1 3 3 3 3 3 1 1 0 1 8 1 0 8 8 0 8 8 1 8 0 3 3 3 8 1 1 0 0 0 0 0 0 0 8 8 0 0 8 0 1 0 0 1 0 0 0 8 1 1 1 0 0 0 1 0 1 0 1 8 8 1 0 0 8 0 1 0 1 1 0 0 0 4 4 4 4 4 8 4 0 0 0 1 0 8 0 8 0 1 8 0 1 4 4 4 4 4 4 4 0 1 1 0 8 0 0 0 0 8 1 8 1 4 4 4 4 4 1 1 0 1 0 1 1 0 0 0 0 1 0 8 0 1 0 0 0 1 8 1 0 8 0 1 0 0 8 0 0 8 1 0 8 0 1 0 0 1 0 8 0 1 1 0 1 8 0 8 0 0 1 0 Output: 3 3 3 3 3 3 3 3 3 Example 3: Input: 0 3 2 0 0 0 0 0 2 0 3 2 2 3 3 2 0 0 0 0 2 2 0 0 2 0 0 0 3 3 2 2 0 3 0 0 3 2 2 3 0 2 8 8 8 8 8 8 0 0 0 2 3 3 0 2 6 6 0 2 3 8 8 8 8 8 8 8 3 0 0 3 2 3 6 6 6 6 6 2 0 8 8 8 8 8 8 8 3 2 0 2 3 9 6 6 6 6 6 3 2 0 8 8 8 8 8 8 8 0 0 2 0 0 6 6 6 6 6 0 0 2 0 8 8 8 8 8 8 2 2 0 6 6 6 6 6 6 6 2 0 0 2 3 8 8 8 8 8 8 0 3 0 6 6 6 6 6 6 3 0 0 0 2 0 0 2 0 0 2 0 3 3 6 6 6 0 0 0 0 0 0 2 2 0 3 0 3 3 0 0 0 0 0 0 0 0 3 0 0 0 2 3 2 2 3 3 2 2 0 2 2 0 2 0 0 2 0 2 2 3 0 0 3 2 0 3 0 0 2 4 4 4 0 0 0 2 0 2 3 0 0 0 0 2 0 2 0 0 4 4 4 4 0 0 3 0 2 0 2 3 2 0 0 0 3 0 0 4 4 4 4 4 3 2 3 2 0 2 0 3 2 2 0 2 0 0 0 4 4 4 4 4 3 2 0 3 0 2 2 2 3 0 0 2 2 0 3 0 4 4 4 4 3 2 0 0 0 3 2 2 2 0 3 0 2 0 3 0 2 3 2 2 2 0 2 2 3 0 3 2 2 0 0 0 0 0 3 2 3 0 2 0 0 0 2 0 2 2 2 0 0 0 0 0 0 3 3 2 3 3 3 2 0 0 3 0 2 3 3 0 0 3 2 0 0 2 2 2 0 3 0 0 2 0 3 0 3 0 0 Output: 0 0 0 0 0 0 0 0 0 Test Input: 0 9 9 7 0 0 9 7 7 7 0 7 0 0 7 0 9 0 0 0 0 0 7 3 3 3 3 0 7 7 0 0 0 0 6 6 6 6 0 9 0 7 9 3 3 3 3 3 3 0 9 9 7 0 6 6 6 6 0 7 0 9 0 3 3 3 3 3 3 3 7 0 7 6 6 6 6 6 0 0 9 0 0 7 0 9 7 7 0 7 7 0 0 0 9 6 6 6 7 7 0 0 9 7 0 9 9 0 0 7 0 0 9 0 0 6 6 6 0 7 0 9 0 9 0 0 7 0 0 9 0 0 0 0 0 9 9 0 0 0 0 9 0 0 9 7 0 0 0 9 7 0 9 9 0 7 0 0 0 0 0 7 8 8 8 8 9 9 0 7 0 0 9 7 7 0 0 9 7 7 9 0 9 8 8 8 7 7 0 7 0 0 9 0 0 9 0 7 0 0 0 0 9 8 8 8 0 9 0 9 0 0 7 5 5 0 0 9 0 9 0 0 9 8 8 8 9 0 0 0 0 9 5 5 5 7 0 0 0 9 9 0 0 8 8 8 0 7 9 9 7 0 5 5 5 5 0 0 0 7 9 9 9 7 9 8 8 0 9 7 0 5 5 5 5 5 9 0 7 0 0 0 7 7 0 7 8 0 0 0 7 5 5 5 5 5 5 9 0 9 9 7 7 0 9 0 7 9 7 0 9 5 5 5 5 5 5 0 0 9 0 7 7 0 0 7 9 0 7 0 9 7 5 5 5 5 5 9 7 9 0 0 7 7 7 0 0 9 0 9 0 0 5 5 5 5 5 0 0 0 0 0 0 0 0 7 9 0 0 0 9 9 5 5 5 0 9 0 9 0 0 0 9 0 7 0 0 9 7 0 0 7 0 0 7 9 0 0 0 0 Analyze the patterns in the training examples and write a Python function that performs this transformation. **Approach Guidelines:** - Look for patterns in shapes, colors, positions, sizes, rotations, reflections, etc. - Even if you can't solve all training examples perfectly, implement what patterns you do observe - A partial solution that captures some aspects is better than returning the input unchanged - If the pattern is unclear, make your best educated guess based on what you can see Requirements: - The function takes a 2D list (grid) where grid[row][col] gives the value at that position - Values are integers from 0-9 - Return a new grid (2D list) with the transformation applied - You can use numpy if needed - just add 'import numpy as np' at the start of your function - Aim to handle the training examples as well as possible, even if not perfectly - Your function should attempt some meaningful transformation based on the patterns you observe You MUST end your response with the following exact format: Final answer: ```python def transform(grid): # Your transformation logic here (implement your best understanding) return transformed_grid ``` "}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize = False,
#     add_generation_prompt = True, # Must add for generation
#     enable_thinking = True, # Disable thinking
# )

# from transformers import TextStreamer
# _ = model.generate(
#     **tokenizer(text, return_tensors = "pt").to("cuda"),
#     max_new_tokens = 1_000, # Increase for longer outputs!
#     temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
#     streamer = TextStreamer(tokenizer, skip_prompt = True),
# )

In [ ]:
# print(tokenizer.chat_template)

In [ ]:
data["validation"]['prompt'][0]

In [ ]:
messages = [
    {"role" : "system", "content" : "You are an expert at solving abstract reasoning puzzles. Write clean, efficient Python code."},
    {"role" : "user", "content" : "You are solving an ARC (Abstraction and Reasoning Corpus) task. \nI will show you training examples with input and output grids, plus a test input grid. Your task is to:\n\n1. **Analyze the training examples** to discover patterns that map input grids to output grids\n2. **Write a Python program** that implements your best understanding of the transformation  \n3. **DO NOT predict or generate the test output** - your job is only to write the transformation program\n4. **Attempt a solution** - even if the pattern isn't completely clear, provide your best hypothesis\n5. **Do not repeat the same transformation** - if you have already tried a transformation, do not repeat it.\n\n**IMPORTANT: Your transformation must always produce a 10\u00d710 output grid.**\n\nThe test input is shown for context so you understand what type of grid your program will eventually process. Focus on learning patterns from training examples and writing code that captures your understanding.\n\nTraining Examples:\n\nExample 1:\nInput:\n5 0 0 5 0 0 0 5 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\nOutput:\n5 0 0 5 0 0 0 5 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n2 0 0 2 0 0 0 2 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n2 0 0 2 0 0 0 2 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n\nExample 2:\nInput:\n0 5 0 5 5 0 0 5 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\nOutput:\n0 5 0 5 5 0 0 5 0 0\n0 0 0 0 0 0 0 0 0 0\n0 2 0 2 2 0 0 2 0 5\n0 0 0 0 0 0 0 0 0 0\n0 2 0 2 2 0 0 2 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 2 0 2 2 0 0 2 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n\nExample 3:\nInput:\n0 0 5 5 0 5 0 5 5 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\nOutput:\n0 0 5 5 0 5 0 5 5 0\n0 0 0 0 0 0 0 0 0 0\n0 0 2 2 0 2 0 2 2 5\n0 0 2 2 0 2 0 2 2 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 2 2 0 2 0 2 2 5\n0 0 0 0 0 0 0 0 0 0\n0 0 2 2 0 2 0 2 2 5\n0 0 0 0 0 0 0 0 0 0\n\nTest Input:\n5 0 5 5 0 0 5 0 5 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n\nAnalyze the patterns in the training examples and write a Python function that performs this transformation.\n\n**Approach Guidelines:**\n- Look for patterns in shapes, colors, positions, sizes, rotations, reflections, etc.\n- Even if you can't solve all training examples perfectly, implement what patterns you do observe\n- A partial solution that captures some aspects is better than returning the input unchanged\n- If the pattern is unclear, make your best educated guess based on what you can see\n\nRequirements:\n- The function takes a 2D list (grid) where grid[row][col] gives the value at that position\n- Values are integers from 0-9\n- Return a new grid (2D list) with the transformation applied\n- You can use numpy if needed - just add 'import numpy as np' at the start of your function\n- Aim to handle the training examples as well as possible, even if not perfectly\n- Your function should attempt some meaningful transformation based on the patterns you observe\n\nYou MUST end your response with the following exact format:\n\nFinal answer:\n```python\ndef transform(grid):\n    # Your transformation logic here (implement your best understanding)\n    return transformed_grid\n```\n"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

# from transformers import TextStreamer
# _ = model.generate(
#     **tokenizer(text, return_tensors = "pt").to("cuda"),
#     max_new_tokens = 8000, # Increase for longer outputs!
#     # temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
#     temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
#     # temperature = 0.01,
#     streamer = TextStreamer(tokenizer, skip_prompt = True),
# )

# text = data["validation"]['prompt'][0]

from transformers import TextStreamer

inputs = tokenizer(text, return_tensors="pt").to("cuda")
input_ids = inputs["input_ids"]  # Extract for convenience

output_ids = model.generate(
    **inputs,
    max_new_tokens=2000,
    # temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    # temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    temperature=0.1, # BEST FOR SINGLE ATTEMPTS
)

# Slice to skip the prompt portion in output
generated_tokens = output_ids[0][input_ids.shape[-1]:]
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(generated_text)

In [ ]:
# Use extract_python_code from utils (SOAR approach)
code = extract_python_code(generated_text)

if code:
    print(code)
    exec(code, globals())  # Defines `transform()` in global scope
else:
    raise ValueError("Could not extract Python code from generated text")

In [ ]:
# def transform(grid):
#     transformed_grid = [row[:] for row in grid]
    
#     # Modify cells in column 2 (index 1) and column 7 (index 6)
#     for row in range(10):
#         # Column 2 (index 1)
#         if grid[row][1] == 0:
#             transformed_grid[row][1] = 2
#         # Column 7 (index 6)
#         if grid[row][6] == 0:
#             transformed_grid[row][6] = 2
    
#     return transformed_grid
# ---
# # CORRECT TRANSFORMATION
# def transform(grid):
#     transformed_grid = [row[:] for row in grid]
#     first_row_pattern = grid[0]
    
#     for r in range(10):
#         if transformed_grid[r][9] == 5:
#             for c in range(9):
#                 if first_row_pattern[c] == 5:
#                     transformed_grid[r][c] = 2
                    
#     return transformed_grid


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# -------------------- helper --------------------
def safe_transform(grid):
    grid = grid.copy()        # <‑‑ clone so the original stays unchanged
    try:
        return transform(grid)
    except Exception as err:
        print(f"[safe_transform] transform() failed – {err}")
        return np.zeros_like(grid)

# -------------------- test case -----------------
test_case = {
    "input": np.array([  # convert to np.array for convenience
        [5, 0, 5, 5, 0, 0, 5, 0, 5, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 5]
    ]),
    "output": np.array([
        [5, 0, 5, 5, 0, 0, 5, 0, 5, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 2, 2, 0, 0, 2, 0, 2, 5],
        [2, 0, 2, 2, 0, 0, 2, 0, 2, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 2, 2, 0, 0, 2, 0, 2, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 2, 2, 0, 0, 2, 0, 2, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 2, 2, 0, 0, 2, 0, 2, 5]
    ])
}

# -------------------- run & plot ----------------
predicted_output = safe_transform(test_case["input"])

fig, axs = plt.subplots(1, 3, figsize=(12, 4))
titles = ["Input", "Predicted Output", "Ground Truth Output"]
grids  = [test_case["input"], predicted_output, test_case["output"]]

for ax, grid, title in zip(axs, grids, titles):
    im = ax.imshow(grid, cmap="viridis", vmin=0, vmax=5)
    ax.set_title(title)
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
stop

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# !pip show transformers

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

You can use this also to load a checkpoint!!! i.e. an intermediate checkpoint from training, so you can then push it to hub.

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        # max_seq_length = 30000,
        load_in_4bit = False,
    )

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
from huggingface_hub import HfFolder, login

# Call this at the top of your script / notebook
if HfFolder.get_token() is None:   # no token cached or in $HF_TOKEN
    login()                        # interactive prompt

In [ ]:
# print(model)

In [ ]:
# run_name = "Qwen2.5-Coder-7B-Instruct-gemini_synth_50_random_split_1_training-20250723-113848"
print(f"Pushing to Trelis/{run_name}")

In [ ]:
model = model.merge_and_unload()
model.push_to_hub(f"Trelis/{run_name}")
tokenizer.push_to_hub(f"Trelis/{run_name}")

In [ ]:
# # Merge to 16bit
# if False:
#     model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
# if True: # Pushing to HF Hub
#     model.push_to_hub_merged(f"Trelis/{run_name}", tokenizer, save_method = "merged_16bit")

# # Merge to 4bit
# if False:
#     model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
# if False: # Pushing to HF Hub
#     model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# # Just LoRA adapters
# if False:
#     model.save_pretrained("model")
#     tokenizer.save_pretrained("model")
# if False: # Pushing to HF Hub
#     model.push_to_hub("hf/model", token = "")
#     tokenizer.push_to_hub("hf/model", token = "")


In [ ]:
# print(tokenizer.chat_template)

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
